In [ ]:
import os
import requests
from typing import Type
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
import wikipedia

# Initialize the LLM
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

# Tool to search Wikipedia
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The topic you want to search on Wikipedia.")

class WikipediaSearchTool(BaseTool):
    name: str = "WikipediaSearchTool"  # Added type annotation
    description: str = "Search for information on Wikipedia."  # Added type annotation
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query: str) -> str:  # Added return type annotation
        try:
            summary = wikipedia.summary(query, sentences=5)
            return summary
        except Exception as e:
            return f"Error while searching Wikipedia: {str(e)}"
        

# Tool to search DuckDuckGo
class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The topic you want to search using DuckDuckGo.")

class DuckDuckGoSearchTool(BaseTool):
    name: str = "DuckDuckGoSearchTool"
    description: str = "Search for information using DuckDuckGo."
    args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query: str) -> str:
        try:
            ddg = DuckDuckGoSearchAPIWrapper()
            results = ddg.results(query)  # 수정: .run()이 아닌 .results()를 사용합니다.
            if results:
                return "\n".join(result["href"] for result in results[:3])  # 상위 3개 링크 반환
            else:
                return "No results found."
        except Exception as e:
            return f"Error while searching DuckDuckGo: {str(e)}"

# Tool to scrape website content
class WebScraperToolArgsSchema(BaseModel):
    url: str = Field(description="The URL of the website to scrape text content from.")

class WebScraperTool(BaseTool):
    name: str = "WebScraperTool"
    description: str = "Scrape text content from a given URL."
    args_schema: Type[WebScraperToolArgsSchema] = WebScraperToolArgsSchema

    def _run(self, url: str) -> str:
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.text[:2000]
        except Exception as e:
            return f"Error while scraping the website: {str(e)}"

# Tool to save research to a .txt file
class SaveToFileToolArgsSchema(BaseModel):
    content: str = Field(description="The content to save to a file.")
    filename: str = Field(description="The name of the file.")

class SaveToFileTool(BaseTool):
    name: str = "SaveToFileTool"
    description: str = "Save research content to a .txt file."
    args_schema: Type[SaveToFileToolArgsSchema] = SaveToFileToolArgsSchema

    def _run(self, content: str, filename: str) -> str:
        try:
            with open(filename, "w", encoding="utf-8") as file:
                file.write(content)
            return f"Content successfully saved to {filename}"
        except Exception as e:
            return f"Error while saving to file: {str(e)}"

# Initialize the agent
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        WebScraperTool(),
        SaveToFileTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
            You are a research assistant.
            Your task is to search for information using Wikipedia and DuckDuckGo, scrape content from relevant websites, and save your findings to a .txt file.
            Always ensure the information is accurate and relevant.
            """
        )
    },
)

# Example usage
query = "Research about the XZ backdoor"

# Step 1: Search Wikipedia
wiki_result = agent.tools[0]._run(query)

# Step 2: Search DuckDuckGo
ddg_result = agent.tools[1]._run(query)

# Step 3: Scrape content from a URL (if found via DuckDuckGo)
scrape_result = ""
if "http" in ddg_result:
    first_url = ddg_result.split("\n")[0]  # Assuming the first line contains a URL
    scrape_result = agent.tools[2]._run(first_url)

# Step 4: Combine results and save to file
combined_content = f"Wikipedia Result:\n{wiki_result}\n\nDuckDuckGo Result:\n{ddg_result}\n\nScraped Content:\n{scrape_result}"
filename = "research_XZ_backdoor.txt"
file_save_result = agent.tools[3]._run(content=combined_content, filename=filename)

print(file_save_result)


C:\Users\xcali\AppData\Local\Temp\ipykernel_5688\3622206279.py:81: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


TypeError: text() got an unexpected keyword argument 'max_results'